In [1]:
import csv
import pandas as pd
import mysql.connector as connector
from datetime import date

In [2]:
dbconn = connector.connect(
    host="localhost",
    user="root",
    password="root1234",
    database="superstore_dataset"
)
cursor = dbconn.cursor()
df = pd.read_csv("../../dataset.csv", encoding='windows-1252')

In [3]:
def get_data(selector):
    result = {}
    select = "SELECT * FROM {}".format(selector)
    try:
        cursor.execute(select)
        data = cursor.fetchall()
        for item in data:
            result[item[1]] = item[0]
    except:
        print("Something went wrong")
        return {}
    return result


In [4]:
def get_fk_id(fk, dict_data):
    arr = []
    for item in dict_data:
        idx = dict_data[item]
        idx = fk[idx]
        arr.append([item, idx])
    return arr

In [5]:
def insert_data(sql_script, data):
    cursor.executemany(sql_script, data)
    try:
        print(cursor.rowcount, "records inserted")
    except:
        print('Something went wrong')
    dbconn.commit()

In [6]:
def insert_id_name(df_key, tb_name):
    dt = df[df_key].unique().tolist()
    arrs = []
    for item in dt:
        arrs.append([item])

    sql = "INSERT INTO {}(name) VALUES(%s)".format(tb_name)
    insert_data(sql, arrs)

In [7]:
data_map = {
    "Country":"countries",
    "Region":"regions",
    "Segment":"segments",
    "Ship Mode":"ship_modes",
    "Category":"categories",
}

for item in data_map:
    insert_id_name(item,data_map[item])

1 records inserted
4 records inserted
3 records inserted
4 records inserted
3 records inserted


In [8]:
def insert_name_idfk(tb_fk, df_fk, tb_name, df_key, fk_attrib):
    res = get_data(tb_fk)
    group = df[[df_key,df_fk]].groupby([df_key]).first().to_dict()
    group = get_fk_id(res, group[df_fk])
    sql = "INSERT INTO {}(name, {}) VALUES(%s,%s)".format(tb_name, fk_attrib)
    insert_data(sql,group)

In [9]:
data_map = [
    ["countries","Country","states","State","country_id"],
    ["states","State","cities","City","state_id"],
    ["categories","Category","sub_categories","Sub-Category","category_id"]
]

for item in data_map:
    insert_name_idfk(item[0],item[1],item[2],item[3],item[4])

49 records inserted
531 records inserted
17 records inserted


In [10]:
def insert_customID_name_fk(tb_fk, df_fk, tb_name, df_key, df_id, fk_attrib):
    res = get_data(tb_fk)

    dt = df[[df_key,df_id,df_fk]].groupby([df_id]).first().to_dict()
    values = {}
    for item in dt:
        for j in dt[item]:
            if item == df_fk:
                dt[item][j] = res[dt[item][j]]
            if j not in values:
                values[j] = [dt[item][j]]
            else:
                values[j].append(dt[item][j])
    results = []
    for item in values:
        temp = values[item]
        item = [item]
        item+=temp
        results.append(item)
    sql = "INSERT INTO {}(id,name,{}) VALUES(%s,%s,%s)".format(tb_name,fk_attrib)
    insert_data(sql, results)
    

In [11]:
data_map = [
    ["segments","Segment","customers","Customer Name","Customer ID","segment_id"],
    ["sub_categories","Sub-Category","products","Product Name","Product ID","sub_category_id"]
]
for item in data_map:
    insert_customID_name_fk(item[0],item[1],item[2],item[3],item[4],item[5])

793 records inserted
1862 records inserted


In [12]:
dt = df[['Order ID','Order Date','Ship Date','Postal Code','Customer ID','Ship Mode','City','Region']].groupby(["Order ID"]).first().to_dict()


# get datas
# customers = get_data("customers")
ship_modes = get_data("ship_modes")
cities = get_data("cities")
regions = get_data("regions")


data_map = {}
for item in dt:
    for order_id in dt[item]:
        if item == "Ship Mode":
            dt[item][order_id] = ship_modes[dt[item][order_id]]
        elif item == "City":
            dt[item][order_id] = cities[dt[item][order_id]]
        elif item == "Region":
            dt[item][order_id] = regions[dt[item][order_id]]
        elif item == "Order Date" or item == "Ship Date":
            temp = dt[item][order_id].split("/")
            dt[item][order_id] = date(eval(temp[2]),eval(temp[0]),eval(temp[1]))
            # print(dt[item][order_id])


        if order_id not in data_map:
            data_map[order_id]=[dt[item][order_id]]
            continue
        data_map[order_id].append(dt[item][order_id])
results = []
for item in data_map:
    temp = data_map[item]
    item = [item]
    item+=temp
    results.append(item)

sql = "INSERT INTO orders(id,order_date,ship_date,postal_code,customer_id,ship_mode_id,city_id,region_id) VALUES(%s,%s,%s,%s,%s,%s,%s,%s)"
insert_data(sql,results)
    

5009 records inserted


In [13]:
dt = df[['Order ID','Sales','Quantity','Discount','Profit','Product ID']].to_dict()

data_map = {}
for item in dt:
    for order_id in dt[item]:
        if order_id not in data_map:
            data_map[order_id]=[dt[item][order_id]]
            continue
        data_map[order_id].append(dt[item][order_id])
results = []
for item in data_map:
    temp = data_map[item]
    results.append(temp)
print(results)

sql = "INSERT INTO order_details(order_id,sales,quantity,discount,profit,product_id) VALUES(%s,%s,%s,%s,%s,%s)"
insert_data(sql,results)
    

[['CA-2016-152156', 261.96, 2, 0.0, 41.9136, 'FUR-BO-10001798'], ['CA-2016-152156', 731.94, 3, 0.0, 219.582, 'FUR-CH-10000454'], ['CA-2016-138688', 14.62, 2, 0.0, 6.8714, 'OFF-LA-10000240'], ['US-2015-108966', 957.5775, 5, 0.45, -383.031, 'FUR-TA-10000577'], ['US-2015-108966', 22.368, 2, 0.2, 2.5164, 'OFF-ST-10000760'], ['CA-2014-115812', 48.86, 7, 0.0, 14.1694, 'FUR-FU-10001487'], ['CA-2014-115812', 7.28, 4, 0.0, 1.9656, 'OFF-AR-10002833'], ['CA-2014-115812', 907.152, 6, 0.2, 90.7152, 'TEC-PH-10002275'], ['CA-2014-115812', 18.504, 3, 0.2, 5.7825, 'OFF-BI-10003910'], ['CA-2014-115812', 114.9, 5, 0.0, 34.47, 'OFF-AP-10002892'], ['CA-2014-115812', 1706.184, 9, 0.2, 85.3092, 'FUR-TA-10001539'], ['CA-2014-115812', 911.424, 4, 0.2, 68.3568, 'TEC-PH-10002033'], ['CA-2017-114412', 15.552, 3, 0.2, 5.4432, 'OFF-PA-10002365'], ['CA-2016-161389', 407.976, 3, 0.2, 132.5922, 'OFF-BI-10003656'], ['US-2015-118983', 68.81, 5, 0.8, -123.858, 'OFF-AP-10002311'], ['US-2015-118983', 2.544, 3, 0.8, -3.816,

In [14]:
dbconn.close()